In [1]:
from itertools import chain
import os

import pandas as pd
pd.options.display.max_colwidth = 500


annotations = pd.read_csv("/Users/olgabot/code/tabula-muris/00_data_ingest/18_global_annotation_csv/annotations_facs.csv")
annotations['cell'] = annotations.cell.str.replace('.', '-')
print(annotations.shape)
annotations.head()

(44949, 24)


/anaconda3/envs/reflow-workflows/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Neurog3>0_raw,Neurog3>0_scaled,cell,cell_ontology_class,cell_ontology_id,cluster.ids,free_annotation,mouse.id,mouse.sex,plate.barcode,...,subsetC,subsetC_cluster.ids,subsetD,subsetD_cluster.ids,subsetE,subsetE_cluster.ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2
0,NaN,NaN,A1-B000610-3_56_F-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_56_F,F,B000610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-7.182841,-10.895094
1,NaN,NaN,A1-B002764-3_38_F-1-1,bladder urothelial cell,CL:1001428,3,Luminal bladder epithelial cell,3_38_F,F,B002764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,4.199059,-11.050240
2,NaN,NaN,A1-B002771-3_39_F-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_39_F,F,B002771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-11.995435,-7.325534
3,NaN,NaN,A1-D041914-3_8_M-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_8_M,M,D041914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-6.820022,-14.174246
4,NaN,NaN,A1-D042253-3_9_M-1-1,bladder cell,CL:1001319,2,Bladder mesenchymal cell,3_9_M,M,D042253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-24.163538,-7.499349


In [2]:
annotations.tissue.value_counts()

Marrow               5037
Fat                  4967
Brain_Myeloid        4455
Heart                4365
Large_Intestine      3938
Brain_Non-Myeloid    3401
Mammary_Gland        2405
Skin                 2310
Lung                 1716
Spleen               1697
Pancreas             1564
Tongue               1416
Bladder              1378
Trachea              1350
Thymus               1349
Limb_Muscle          1090
Diaphragm             870
Liver                 714
Kidney                519
Aorta                 408
Name: tissue, dtype: int64

In [3]:
marrow_kidney = "Marrow", "Kidney"
annotations_marrow_kidney = annotations.query('tissue in @marrow_kidney')
print(annotations_marrow_kidney.shape)
annotations_marrow_kidney.head()

(5556, 24)


,Neurog3>0_raw,Neurog3>0_scaled,cell,cell_ontology_class,cell_ontology_id,cluster.ids,free_annotation,mouse.id,mouse.sex,plate.barcode,...,subsetC,subsetC_cluster.ids,subsetD,subsetD_cluster.ids,subsetE,subsetE_cluster.ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2
15122,NaN,NaN,A1-B001717-3_38_F-1-1,macrophage,CL:0000235,8,NaN,3_38_F,F,B001717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,-21.029031,-1.596872
15123,NaN,NaN,A10-B002775-3_39_F-1-1,endothelial cell,CL:0000115,0,NaN,3_39_F,F,B002775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,-2.404816,-16.146999
15124,NaN,NaN,A10-MAA000752-3_10_M-1-1,endothelial cell,CL:0000115,5,NaN,3_10_M,M,MAA000752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,-3.066679,-13.670210
15125,NaN,NaN,A11-MAA000801-3_11_M-1-1,kidney collecting duct epithelial cell,CL:1000454,1,NaN,3_11_M,M,MAA000801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,11.566742,-3.721952
15126,NaN,NaN,A12-B001717-3_38_F-1-1,macrophage,CL:0000235,8,NaN,3_38_F,F,B001717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,-21.595881,-1.731221


In [4]:
annotations_marrow_kidney.tissue.value_counts()

Marrow    5037
Kidney     519
Name: tissue, dtype: int64

In [5]:
# %%time

# ! aws s3 ls --recursive s3://czb-maca/Plate_seq/3_month/ | grep -E 'fastq.gz$' > tabula_muris_3month_fastqs.txt

In [6]:
bucket = 's3://czb-maca/'

maca_aws = pd.read_table('tabula_muris_3month_fastqs.txt', sep='\s+', header=None, 
                         names=['date', 'time', 'filesize', 'suffix'])
maca_aws['megabytes'] = maca_aws['filesize']/1e6
maca_aws['novaseq_run'] = maca_aws['suffix'].str.extract('([AB]\d{5})')
maca_aws['run_id'] = maca_aws['suffix'].str.split('/').str[2]
maca_aws['novaseq_run_number'] = maca_aws['novaseq_run'].str.lstrip("AB")
maca_aws['full_path'] = bucket + maca_aws['suffix']
maca_aws['read_number'] = maca_aws.suffix.str.extract('_(R[12])_')
maca_aws['sample_id'] = maca_aws.suffix.map(lambda x: os.path.basename(x).split("_R")[0])
print(maca_aws.shape)

maca_aws.head()

/anaconda3/envs/reflow-workflows/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: read_table is deprecated, use read_csv instead.


(319488, 11)


,date,time,filesize,suffix,megabytes,novaseq_run,run_id,novaseq_run_number,full_path,read_number,sample_id
0,2018-08-23,12:54:15,53806,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041912-3_8_M-1-1_R1_001.fastq.gz,0.053806,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041912-3_8_M-1-1_R1_001.fastq.gz,R1,A1-D041912-3_8_M-1-1
1,2018-08-23,12:54:16,54967,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041912-3_8_M-1-1_R2_001.fastq.gz,0.054967,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041912-3_8_M-1-1_R2_001.fastq.gz,R2,A1-D041912-3_8_M-1-1
2,2018-08-23,12:54:17,36204,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041914-3_8_M-1-1_R1_001.fastq.gz,0.036204,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041914-3_8_M-1-1_R1_001.fastq.gz,R1,A1-D041914-3_8_M-1-1
3,2018-08-23,12:54:17,36781,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041914-3_8_M-1-1_R2_001.fastq.gz,0.036781,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041914-3_8_M-1-1_R2_001.fastq.gz,R2,A1-D041914-3_8_M-1-1
4,2018-08-23,12:54:18,18552,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D042479-3_8_M-1-1_R1_001.fastq.gz,0.018552,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D042479-3_8_M-1-1_R1_001.fastq.gz,R1,A1-D042479-3_8_M-1-1


In [7]:
maca_aws.run_id.unique()

array(['170907_A00111_0051_BH2HWLDMXX', '170907_A00111_0052_AH2HTCDMXX',
       '170910_A00111_0053_BH2HGKDMXX', '170910_A00111_0054_AH2HGWDMXX',
       '170910_A00111_0054_AH2HGWDMXX__170910_A00111_0053_BH2HGKDMXX',
       '170914_A00111_0057_BH3FY7DMXX', '170914_A00111_0058_AH3FYKDMXX',
       '170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX',
       '170918_A00111_0059_BH3G22DMXX', '170918_A00111_0060_AH3FYVDMXX',
       '170918_A00111_0060_AH3FYVDMXX__170918_A00111_0059_BH3G22DMXX',
       '170921_A00111_0062_BH3FYHDMXX', '170921_A00111_0063_AH3G23DMXX',
       '170921_A00111_0063_AH3G23DMXX__170921_A00111_0062_BH3FYHDMXX',
       '170925_A00111_0066_AH3TKNDMXX',
       '170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX',
       '170925_A00111_0067_BH3M5YDMXX', '170928_A00111_0068_AH3YKKDMXX',
       '170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX',
       '170928_A00111_0069_BH52YMDMXX'], dtype=object)

The ones containing '__' are combined runs

In [8]:


combined_runs = list(chain(*[x.split('__') for x in maca_aws.run_id.unique() if '__' in x]))
combined_runs

['170910_A00111_0054_AH2HGWDMXX',
 '170910_A00111_0053_BH2HGKDMXX',
 '170914_A00111_0058_AH3FYKDMXX',
 '170914_A00111_0057_BH3FY7DMXX',
 '170918_A00111_0060_AH3FYVDMXX',
 '170918_A00111_0059_BH3G22DMXX',
 '170921_A00111_0063_AH3G23DMXX',
 '170921_A00111_0062_BH3FYHDMXX',
 '170925_A00111_0066_AH3TKNDMXX',
 '170925_A00111_0067_BH3M5YDMXX',
 '170928_A00111_0068_AH3YKKDMXX',
 '170928_A00111_0069_BH52YMDMXX']

In [9]:
maca_aws_no_uncombined = maca_aws.query('run_id not in @combined_runs')
print(maca_aws_no_uncombined.shape)
maca_aws_no_uncombined.head()

(107520, 11)


,date,time,filesize,suffix,megabytes,novaseq_run,run_id,novaseq_run_number,full_path,read_number,sample_id
0,2018-08-23,12:54:15,53806,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041912-3_8_M-1-1_R1_001.fastq.gz,0.053806,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041912-3_8_M-1-1_R1_001.fastq.gz,R1,A1-D041912-3_8_M-1-1
1,2018-08-23,12:54:16,54967,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041912-3_8_M-1-1_R2_001.fastq.gz,0.054967,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041912-3_8_M-1-1_R2_001.fastq.gz,R2,A1-D041912-3_8_M-1-1
2,2018-08-23,12:54:17,36204,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041914-3_8_M-1-1_R1_001.fastq.gz,0.036204,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041914-3_8_M-1-1_R1_001.fastq.gz,R1,A1-D041914-3_8_M-1-1
3,2018-08-23,12:54:17,36781,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041914-3_8_M-1-1_R2_001.fastq.gz,0.036781,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D041914-3_8_M-1-1_R2_001.fastq.gz,R2,A1-D041914-3_8_M-1-1
4,2018-08-23,12:54:18,18552,Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D042479-3_8_M-1-1_R1_001.fastq.gz,0.018552,A00111,170907_A00111_0051_BH2HWLDMXX,00111,s3://czb-maca/Plate_seq/3_month/170907_A00111_0051_BH2HWLDMXX/fastqs/A1-D042479-3_8_M-1-1_R1_001.fastq.gz,R1,A1-D042479-3_8_M-1-1


In [12]:
fastqs = maca_aws_no_uncombined.pivot(index='sample_id', columns='read_number', values='full_path')
print(fastqs.shape)
fastqs.head()

(53760, 2)


read_number,R1,R2
sample_id,,
A1-B000126-3_39_F-1-1,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000126-3_39_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000126-3_39_F-1-1_R2_001.fastq.gz
A1-B000127-3_38_F-1-1,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000127-3_38_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000127-3_38_F-1-1_R2_001.fastq.gz
A1-B000167-3_56_F-1-1,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000167-3_56_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000167-3_56_F-1-1_R2_001.fastq.gz
A1-B000168-3_57_F-1-1,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000168-3_57_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000168-3_57_F-1-1_R2_001.fastq.gz
A1-B000412-3_56_F-1-1,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000412-3_56_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000412-3_56_F-1-1_R2_001.fastq.gz


In [13]:
from s3_utils import write_s3

In [14]:
write_s3?

In [15]:
write_s3(fastqs, "s3://czb-maca/Plate_seq/3_month/fastqs.csv")

In [16]:
fastqs_annotated = fastqs.merge(annotations, left_index=True, right_on='cell')
print(fastqs_annotated.shape)
fastqs_annotated.head()

(44949, 26)


,R1,R2,Neurog3>0_raw,Neurog3>0_scaled,cell,cell_ontology_class,cell_ontology_id,cluster.ids,free_annotation,mouse.id,...,subsetC,subsetC_cluster.ids,subsetD,subsetD_cluster.ids,subsetE,subsetE_cluster.ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2
9746,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000126-3_39_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000126-3_39_F-1-1_R2_001.fastq.gz,NaN,NaN,A1-B000126-3_39_F-1-1,epidermal cell,CL:0000362,4,Intermediate IFE,3_39_F,...,NaN,NaN,NaN,NaN,NaN,NaN,Telogen,Skin,30.769229,7.454868
4779,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000127-3_38_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000127-3_38_F-1-1_R2_001.fastq.gz,NaN,NaN,A1-B000127-3_38_F-1-1,endothelial cell,CL:0000115,2,NaN,3_38_F,...,NaN,NaN,NaN,NaN,NaN,NaN,SCAT,Fat,-0.820131,7.154154
25515,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000167-3_56_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170925_A00111_0066_AH3TKNDMXX__170925_A00111_0067_BH3M5YDMXX/fastqs/A1-B000167-3_56_F-1-1_R2_001.fastq.gz,NaN,NaN,A1-B000167-3_56_F-1-1,basal cell,CL:0000646,2,NaN,3_56_F,...,NaN,NaN,NaN,NaN,NaN,NaN,Mammary_Gland,Mammary_Gland,-3.795372,2.943592
40176,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000412-3_56_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000412-3_56_F-1-1_R2_001.fastq.gz,NaN,NaN,A1-B000412-3_56_F-1-1,endothelial cell,CL:0000115,2,NaN,3_56_F,...,NaN,NaN,NaN,NaN,NaN,NaN,RA,Heart,-29.766385,-4.010628
0,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000610-3_56_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A1-B000610-3_56_F-1-1_R2_001.fastq.gz,NaN,NaN,A1-B000610-3_56_F-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_56_F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-7.182841,-10.895094


In [17]:
kidney = fastqs_annotated.query('tissue == "Kidney"')
print(kidney.shape)
kidney.head()

(519, 26)


,R1,R2,Neurog3>0_raw,Neurog3>0_scaled,cell,cell_ontology_class,cell_ontology_id,cluster.ids,free_annotation,mouse.id,...,subsetC,subsetC_cluster.ids,subsetD,subsetD_cluster.ids,subsetE,subsetE_cluster.ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2
15122,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A1-B001717-3_38_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A1-B001717-3_38_F-1-1_R2_001.fastq.gz,NaN,NaN,A1-B001717-3_38_F-1-1,macrophage,CL:0000235,8,NaN,3_38_F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,-21.029031,-1.596872
15123,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A10-B002775-3_39_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A10-B002775-3_39_F-1-1_R2_001.fastq.gz,NaN,NaN,A10-B002775-3_39_F-1-1,endothelial cell,CL:0000115,0,NaN,3_39_F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,-2.404816,-16.146999
15124,s3://czb-maca/Plate_seq/3_month/170910_A00111_0054_AH2HGWDMXX__170910_A00111_0053_BH2HGKDMXX/fastqs/A10-MAA000752-3_10_M-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170910_A00111_0054_AH2HGWDMXX__170910_A00111_0053_BH2HGKDMXX/fastqs/A10-MAA000752-3_10_M-1-1_R2_001.fastq.gz,NaN,NaN,A10-MAA000752-3_10_M-1-1,endothelial cell,CL:0000115,5,NaN,3_10_M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,-3.066679,-13.670210
15125,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A11-MAA000801-3_11_M-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A11-MAA000801-3_11_M-1-1_R2_001.fastq.gz,NaN,NaN,A11-MAA000801-3_11_M-1-1,kidney collecting duct epithelial cell,CL:1000454,1,NaN,3_11_M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,11.566742,-3.721952
15126,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A12-B001717-3_38_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A12-B001717-3_38_F-1-1_R2_001.fastq.gz,NaN,NaN,A12-B001717-3_38_F-1-1,macrophage,CL:0000235,8,NaN,3_38_F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kidney,-21.595881,-1.731221


In [19]:

mouse_samples = kidney[['cell', 'R1', 'R2']]
mouse_samples = mouse_samples.rename(columns={'cell': "sample_id", 'R1': "read1", 'R2': 'read2'})
mouse_samples = mouse_samples.set_index('sample_id')
print(mouse_samples.shape)
mouse_samples.head()

(519, 2)


,read1,read2
sample_id,,
A1-B001717-3_38_F-1-1,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A1-B001717-3_38_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A1-B001717-3_38_F-1-1_R2_001.fastq.gz
A10-B002775-3_39_F-1-1,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A10-B002775-3_39_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170914_A00111_0058_AH3FYKDMXX__170914_A00111_0057_BH3FY7DMXX/fastqs/A10-B002775-3_39_F-1-1_R2_001.fastq.gz
A10-MAA000752-3_10_M-1-1,s3://czb-maca/Plate_seq/3_month/170910_A00111_0054_AH2HGWDMXX__170910_A00111_0053_BH2HGKDMXX/fastqs/A10-MAA000752-3_10_M-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170910_A00111_0054_AH2HGWDMXX__170910_A00111_0053_BH2HGKDMXX/fastqs/A10-MAA000752-3_10_M-1-1_R2_001.fastq.gz
A11-MAA000801-3_11_M-1-1,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A11-MAA000801-3_11_M-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A11-MAA000801-3_11_M-1-1_R2_001.fastq.gz
A12-B001717-3_38_F-1-1,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A12-B001717-3_38_F-1-1_R1_001.fastq.gz,s3://czb-maca/Plate_seq/3_month/170928_A00111_0068_AH3YKKDMXX__170928_A00111_0069_BH52YMDMXX/fastqs/A12-B001717-3_38_F-1-1_R2_001.fastq.gz


In [26]:
from aguamenti.s3_utils import get_fastqs_as_r1_r2_columns

buckets = ('s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/', 's3://olgabot-maca/sra/danio_rerio/smart-seq/whole_kidney_marrow_prjna393431/')

dfs = []

for bucket in buckets:
    df = get_fastqs_as_r1_r2_columns(s3_input_path=bucket)
    print(bucket, df.shape)
    dfs.append(df)
human_zebrafish_samples = pd.concat(dfs)
print(human_zebrafish_samples.shape)
human_zebrafish_samples.index.name = 'sample_id'
human_zebrafish_samples.head()

0it [00:00, ?it/s]

Recursively searching s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/ for fastq.gz files ...


5048it [00:01, 2575.65it/s]
0it [00:00, ?it/s]

	Done. Found 3786 fastq.gz files
	Done. Found 1262 samples' reads (single or paired)
s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/ (1262, 2)
Recursively searching s3://olgabot-maca/sra/danio_rerio/smart-seq/whole_kidney_marrow_prjna393431/ for fastq.gz files ...


738it [00:00, 1497.46it/s]

	Done. Found 492 fastq.gz files
	Done. Found 246 samples' reads (single or paired)
s3://olgabot-maca/sra/danio_rerio/smart-seq/whole_kidney_marrow_prjna393431/ (246, 2)
(1508, 2)


read_number,read1,read2
sample_id,,
SRR4050379_pass,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050379_pass_1.fastq.gz,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050379_pass_2.fastq.gz
SRR4050380_pass,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050380_pass_1.fastq.gz,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050380_pass_2.fastq.gz
SRR4050381_pass,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050381_pass_1.fastq.gz,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050381_pass_2.fastq.gz
SRR4050382_pass,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050382_pass_1.fastq.gz,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050382_pass_2.fastq.gz
SRR4050383_pass,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050383_pass_1.fastq.gz,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050383_pass_2.fastq.gz


In [27]:
samples = pd.concat([human_zebrafish_samples, mouse_samples])
samples.to_csv('/Users/olgabot/code/nf-kmer-similarity/samples.csv')

In [28]:
! head '/Users/olgabot/code/nf-kmer-similarity/samples.csv'

sample_id,read1,read2
SRR4050379_pass,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050379_pass_1.fastq.gz,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050379_pass_2.fastq.gz
SRR4050380_pass,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050380_pass_1.fastq.gz,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050380_pass_2.fastq.gz
SRR4050381_pass,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050381_pass_1.fastq.gz,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050381_pass_2.fastq.gz
SRR4050382_pass,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd38+_bone_marrow_prjna304408/SRR4050382_pass_1.fastq.gz,s3://olgabot-maca/sra/homo_sapiens/smartseq2_quartzseq/lin-cd34+cd3